<a href="https://colab.research.google.com/github/Calanga88/Calanga88/blob/main/SIGE_1_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import...

In [ ]:
import re
import json
import hashlib
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
host = 'sigeur.up.ac.mz'
#sigeur.up.ac.mz
#host = 'sigeul.unilicungo.ac.mz'
# Crie uma sessão
#session = requests.Session()

cookies = {
    'PHPSESSID': 'lfanvu8vje37lp75v9hn2g4efa',
    '_ga': 'GA1.3.572647773.1697004062',
    '_gid': 'GA1.3.1626586042.1697004062',
    '_gat_LoginPage': '1',
}

headers = {
    'Host': f'{host}',
    # 'Content-Length': '35',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Sec-Ch-Ua-Mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.141 Safari/537.36',
    'Sec-Ch-Ua-Platform': '""',
    'Origin': f'https://{host}',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': f'https://{host}/sigeup/',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-PT,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'close',
    # 'Cookie': 'PHPSESSID=t7a1thdtq79gv9ou3442uvr7a7; _ga=GA1.3.572647773.1697004062; _gid=GA1.3.1626586042.1697004062; _gat_LoginPage=1',
}


In [ ]:
def login(usuario, senha):
  data = {
      'usuario': usuario,
      'senha': senha,
  }

  response = requests.post(
      f'https://{host}/edondzo/controller/login.php',
      cookies=cookies,
      headers=headers,
      data=data,
      verify=False,
  )
  print(response.text)


#usuario='mmia'
#senha='Fernando01'
#usuario=ahenriqueta&senha=1904neves
usuario='oadamo'
senha='LAURAALIFA2023'
login(usuario, senha)

"sucesso"


#Functions

In [ ]:
#executa comandos sql (INSERT, UPDATE nd DELETE)
def cmd_sql(sql_update):

  #sql_update = f"UPDATE inscricao_valor set valor={val} where estudante_id = '{estudante_id}' and ano_lectivo_id = (select ano_lectivo_id from inscricao where id='{inscricao_id}') and semestre= (select semestre from inscricao where id='{inscricao_id}')"
  data = f"criterio=delete&id=00000';{sql_update};--"

  response = requests.post(
        f'https://{host}/edondzo/controller/instituicao.php',
        cookies=cookies,
        headers=headers,
        data=data,
        verify=False,
    )

  soup = BeautifulSoup(response.text, "html.parser")
  h4 = soup.find_all('h4')
  v = 287
  lst = []
  for h4_t in h4:
    if h4_t.getText() != '':
        lst.append(h4_t.getText())

  if len(lst) == 0:
      messege = '(Sucesso) SQL Update'

  else:
      messege = f'(Erro ao gravar) SQL Update: {lst}'

  print(messege)

#executa comandos de consulta
def SELECT(consulta):

  data = f"inicio=0&id=&nome=&campus=&curso={consulta};--"

  response = requests.post(
      f'https://{host}/edondzo/view/estudante_lista.php',
      cookies=cookies,
      headers=headers,
      data=data,
      verify=False,
  )
  soup = BeautifulSoup(response.text, "html.parser")
  p_tag = soup.find_all('td')
  #print(soup)
  dados = []
  for p in p_tag:
    if p.getText() != '':
        dados.append(p.getText())

  del dados[-1]

  try:
    #print(type(dados))
    return dados
  except Exception as e:
    print('Log:', e)

def parse_cookie_and_session(text):
  cookie = re.search(r'cookie=(.*?)&', text).group(1)
  session = re.search(r'sessao=(.*)', text).group(1)
  return {
    'cookie': cookie,
    'session': session,
  }

def md5Hash(string):
  # initializing string
  str2hash = string
  print(string)

  # encoding GeeksforGeeks using encode()
  # then sending to md5()
  result = hashlib.md5(str2hash.encode())
  return result.hexdigest()

def list_to_dic(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

def repeat(word, m, n, id_max):
    if(m > len(word)):
        m = len(word)

    id_max += 1
    repeat_word = word[:m]
    result = f"{id_max}"

    for i in range(n):
        result = repeat_word+result

    return result

def get_diciplinas(ano, estudante_id, all: bool = False) -> list:
  if not all:
    sql_cmd =f"'UNION+SELECT+disciplina_id,docente,nome_disciplina,null,turma_id,nome_disciplina,null,null,null+FROM+pauta+where+ano_lectivo_id='{ano}'+and+estudante_id='{estudante_id}'+and+resultado_final+!='Aprovado' and resultado_final+!='Participou';--"
  else:
    #consulta as diciplinas de acordo com o ano lectivo (incluindo as aprovadas)
    sql_cmd =f"'UNION+SELECT+disciplina_id,docente,nome_disciplina,null,turma_id,nome_disciplina,null,null,null+FROM+pauta+where+ano_lectivo_id='{ano}'+and+estudante_id='{estudante_id}';--"
  lista = SELECT(sql_cmd)
  #print(lista)
  return Convert(lista)

In [ ]:
def valida_pagamento_inscr(ano_lectivo_id, semestre, estudante_id, confirmada : bool = False):
  #verifica se e estudante de 2024, se for definir eid_banco estudante antigo, caso nao eid_banco = estudante_id
  banco_id_dic = {'06': '06.0485.2023', '07': '07.0940.2020', '08': '08.0189.2021', '05': '05.0485.2023', '01':'01.0940.2020', '04':'04.0189.2021'}
  if estudante_id[8:] == ano_lectivo_id:
    eid_banco = banco_id_dic[estudante_id[:2]]
  else:
    eid_banco = estudante_id

  #Insere dados de recibo
  sqlCmd = f"""
  WITH DatasINSCR AS (
      SELECT referencia, id, ano_lectivo_id, semestre
      FROM temp_inscricao
      WHERE ano_lectivo_id ='{ano_lectivo_id}' AND semestre = '{semestre}' AND confirmada = '0' AND estudante_id = '{estudante_id}'
  ),
  DatasCapturada AS (
      SELECT data_comprovativo, data_emissao, trace_id
      FROM recibo
      WHERE num_comprovativo = (SELECT referencia FROM inscricao i
          INNER JOIN recibo r ON i.referencia = r.num_comprovativo
          WHERE ano_lectivo_id = (SELECT ano_lectivo_id FROM DatasINSCR) AND semestre = (SELECT semestre FROM DatasINSCR)
          LIMIT 1)
  )
  INSERT INTO recibo (id, estudante_id, data_emissao, banco, forma_pagto, num_comprovativo, data_comprovativo, valor, extenso, trace_id, key)
  SELECT '2020080000000' || (SELECT nextval('seq_recibo')::text), ti.estudante_id, (SELECT data_emissao FROM DatasCapturada), (SELECT MAX(banco) FROM recibo WHERE estudante_id = '{eid_banco}'), 'Servicos', ti.referencia, (SELECT data_comprovativo FROM DatasCapturada), ti.valor, (select extenso from recibo where valor = ti.valor limit 1), (select trace_id from DatasCapturada), (select key from recibo limit 1)
  FROM temp_inscricao ti
  WHERE ti.referencia = (SELECT referencia FROM DatasINSCR)"""
  print('Inserindo dados de recibo: ', estudante_id)
  cmd_sql(sqlCmd)

  #--insere dados na tabela recebimento (comando unico)
  sqlCmd = f"""
  WITH ReferenciaCapturada AS (
      SELECT referencia, id, ano_lectivo_id, semestre
      FROM temp_inscricao
      WHERE ano_lectivo_id ='{ano_lectivo_id}' AND semestre = '{semestre}' AND confirmada = '0' AND estudante_id = '{estudante_id}'
  ),
  DataCapturada AS (
      SELECT data_movimento
      FROM recebimento
      WHERE recibo_id = (SELECT id FROM recibo where num_comprovativo = (SELECT referencia FROM inscricao i
          INNER JOIN recibo r ON i.referencia = r.num_comprovativo
          WHERE ano_lectivo_id = (SELECT ano_lectivo_id FROM ReferenciaCapturada) AND semestre = (SELECT semestre FROM ReferenciaCapturada)
          LIMIT 1)) LIMIT 1
  )
  INSERT INTO recebimento (id, tipo, estudante_id, historico, valor, recibo_id, trace_id, data_movimento)
  SELECT '2023000000000' || (SELECT tp.id::text), 'Emolumento', ti.estudante_id, tp.nome, tp.valor, r.id, r.trace_id, (select data_movimento from DataCapturada) FROM temp_inscricao ti, recibo r, temp_pagamentos tp where ti.referencia = (select referencia from ReferenciaCapturada) AND r.num_comprovativo = (select referencia from ReferenciaCapturada) AND tp.temp_inscricao_id = (select id from ReferenciaCapturada)"""
  print('Inserindo dados na tabela recebimento (comando unico): ', estudante_id)
  cmd_sql(sqlCmd)

  #--Insere dados na tabela inscricao_valor
  sqlCmd = f"""
  WITH ReferenciaCapturada AS (
      SELECT referencia, id, ano_lectivo_id, semestre
      FROM temp_inscricao
      WHERE ano_lectivo_id ='{ano_lectivo_id}' AND semestre = '{semestre}' AND confirmada = '0' AND estudante_id = '{estudante_id}'
  )
  INSERT INTO inscricao_valor (estudante_id, ano_lectivo_id, semestre, valor, trace_id, referencia, key)
                              (SELECT ti.estudante_id, ti.ano_lectivo_id, ti.semestre, r.valor, r.trace_id, r.id, (select key from recibo limit 1) FROM temp_inscricao ti INNER JOIN recibo r ON r.num_comprovativo = ti.referencia where ti.id = (select id from ReferenciaCapturada))"""
  print('Inserindo dados na tabela inscricao_valor: ', estudante_id)
  cmd_sql(sqlCmd)

In [ ]:
def getPre(estudante_id,ano_lectivo_id,semestre):
  data = f"inicio=0&id=&nome=&campus=&curso='UNION+SELECT+id,null,null,null,null,null,null,null,null FROM temp_inscricao where estudante_id ='{estudante_id}' and ano_lectivo_id = '{ano_lectivo_id}' and semestre='{semestre}' and confirmada != 1;--"

  response = requests.post(
    f'https://{host}/edondzo/view/estudante_lista.php',
    cookies=cookies,
    headers=headers,
    data=data,
    verify=False,
  )

  soup = BeautifulSoup(response.text, "html.parser")
  dd = soup.find_all('tbody')
  id_tag = soup.find_all('td')

  #print(soup)
  list_id = []
  for d in id_tag:
      if d.getText() != '':
          list_id.append(d.getText())
  list_id.remove('\nAnterior\nPróximo\n')

  try:
    pre = list_id[0]
    return pre
  except IndexError:
    print('Estudante nao tem pre ou ja esta Inscrito')
    return None

def getLogs_():
  data = {
    'log': 'enviarDadosParaLogInscricaoDoEstudanteNoSistema',
    'log2': 'receberDadosDeConfirmacaoconfirmarInscricaoOnlineparaContinuar',
  }

  response = requests.post(
      f'https://{host}/sigeup/app/helpers/index.php',
      cookies=cookies,
      headers=headers,
      data=data,
      verify=False,
  )
  return response.text

def confirma_inscr(inscricao_id):
  cookie_and_session = parse_cookie_and_session(getLogs_())
  #print(cookie_and_session)
  data = {
      'cookie': f"{cookie_and_session['cookie']}",
      'sessao': f"{cookie_and_session['session']}",
      'id_inscricao': f'{inscricao_id}',
      'intencao': 'undefined',
  }

  try:
    print(f'Host: https://{host}/sigeup/public/index.php')
    response = requests.post(
        f'https://{host}/sigeup/public/index.php',
        cookies=cookies,
        headers=headers,
        data=data,
        verify=False,
    )
    mensagem = json.loads(response.text)
    soup = BeautifulSoup(mensagem['mensagem'], "html.parser")
    h4 = soup.find('h4')
    li = soup.find('li')
    print(h4.getText(), li.getText())
  except Exception as e:
    print('Error:', e)


In [ ]:
def Convert(lista):
  # Inicializa a lista de dicionários vazia
  lista_de_dicts = []

  # Itera sobre a lista de dados de três em três elementos
  for disciplina_id, docente, turma_id, resultado in zip(lista[::4], lista[1::4], lista[2::4], lista[3::4]):
    # Cria um novo dicionário com as informações da disciplina, docente, turma e resultado
    novo_dict = {'disciplina_id': disciplina_id, 'docente': docente, 'turma_id': turma_id, 'nome_disciplina': resultado}
    # Adiciona o novo dicionário à lista de dicionários se o resultado for Reprovado
    #if resultado == 'Reprovado':
    lista_de_dicts.append(novo_dict)
  return lista_de_dicts

In [ ]:
def sum_debito(years: list, estudante_id):
  sql_cmd =f"'UNION+SELECT+CAST(SUM(debito) AS VARCHAR),null,null,null,null,null,null,null,null+FROM conta_corrente WHERE extract(year from data_debito) in {tuple(years)} AND extract(month from data_debito) <= 12 AND upper(estudante_id) = '{estudante_id}' AND CAST(credito AS varchar) != CAST(debito AS varchar);--"
  #print(sql_cmd)
  print(SELECT(sql_cmd))

def getListProp(years: list, estudante_id, value: bool = False, h = 'null'):
  if value:
    v = 'CAST(debito AS VARCHAR)'
  else:
    v = 'null'
  sql_cmd =f"'UNION+SELECT+id,{v},null,null,null,{h},null,null,null+FROM conta_corrente WHERE extract(year from data_debito) in {tuple(years)} AND extract(month from data_debito) <= 12 AND upper(estudante_id) = '{estudante_id}' AND CAST(credito AS varchar) != CAST(debito AS varchar);--"
  props = SELECT(sql_cmd)
  return props

def update_conta_corrente(propid, estudante_id, year, semestre):
  #{estudante_id}
  sqlProp = f"""
    WITH DatasMensa AS (
      SELECT referencia_bancaria
      FROM conta_corrente
      WHERE id = '{propid}'
    ),
    DatasCapturada AS (
        SELECT data_comprovativo, data_emissao, trace_id
        FROM recibo
        WHERE num_comprovativo = (SELECT referencia FROM inscricao i
            INNER JOIN recibo r ON i.referencia = r.num_comprovativo
            WHERE ano_lectivo_id = '{year}' AND semestre = '{semestre}'
            LIMIT 1)
    )
    INSERT INTO recibo (id, estudante_id, data_emissao, banco, forma_pagto, num_comprovativo, data_comprovativo, valor, extenso, trace_id)
    SELECT '2020080000000' || (SELECT nextval('seq_recibo')::text), cc.estudante_id, (SELECT data_emissao FROM DatasCapturada), (SELECT MIN(banco) FROM recibo WHERE estudante_id = '06.1320.2022'), 'Servicos', cc.referencia_bancaria, (SELECT data_comprovativo FROM DatasCapturada), cc.debito, (select extenso from recibo where valor = cc.debito limit 1), (select trace_id from DatasCapturada)
    FROM conta_corrente cc
    WHERE cc.referencia_bancaria = (SELECT referencia_bancaria FROM DatasMensa)"""
  print('Inserindo dados de recibo: ', propid)
  cmd_sql(sqlProp)
  #print(sqlProp)

  sqlProp = f"""
    WITH ReferenciaCapturada AS (
      SELECT referencia_bancaria
      FROM conta_corrente
      WHERE id = '{propid}'
    ),
    DataCapturada AS (
        SELECT data_movimento
        FROM recebimento
        WHERE recibo_id = (SELECT id FROM recibo where num_comprovativo = (SELECT referencia FROM inscricao i
            INNER JOIN recibo r ON i.referencia = r.num_comprovativo
            WHERE ano_lectivo_id = '{year}' AND semestre = '{semestre}'
            LIMIT 1)) LIMIT 1
    )
    INSERT INTO recebimento (id, tipo, estudante_id, historico, valor, recibo_id, trace_id, data_movimento)
    SELECT '2023000000000' || (SELECT nextval('seq_recebimento')::text), 'Propina', cc.estudante_id, cc.historico, cc.debito, r.id, r.trace_id, (select data_movimento from DataCapturada) FROM conta_corrente cc, recibo r where cc.referencia_bancaria = (select referencia_bancaria from ReferenciaCapturada) AND r.num_comprovativo = (select referencia_bancaria from ReferenciaCapturada)"""
  print('Inserindo dados na tabela recebimento (comando unico): ', propid)
  cmd_sql(sqlProp)
  #print(sqlProp)

def valida_props(year, estudante_id, propids):
  if len(propids) > 1:
    list_ids = tuple(propids)
  else:
    list_ids = f"('{propids[0]}')"
  sqlProp = f"""
    WITH UlccmoRegistro2023 AS (
      SELECT MAX(id) AS ulccmo_id
      FROM conta_corrente
      WHERE extract(year from data_debito) = {year}
    )
    UPDATE conta_corrente AS cc
    SET credito = debito,
        trace_id = (SELECT trace_id FROM conta_corrente WHERE id = (SELECT ulccmo_id FROM UlccmoRegistro2023)),
        data_pagto = (SELECT data_debito FROM conta_corrente WHERE id = (SELECT ulccmo_id FROM UlccmoRegistro2023))
    WHERE id IN {list_ids}
        AND upper(estudante_id) = '{estudante_id}'
        AND CAST(credito AS varchar) != CAST(debito AS varchar)"""
  cmd_sql(sqlProp)
  print("Pedido executado com sucesso, Mensalidades validadas com Sucesso!")

#Confirma inscr

In [ ]:
# prompt: execute a funcao valida pagamento
ano_lectivo_id = '2024'
semestre= '1'
estudantes = [
    '06.2288.2024',
    '06.2289.2024']
    #'06.0284.2018'

cmd_sql("ALTER TABLE recibo DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE inscricao_valor DISABLE TRIGGER key_check_on")
for estudante_id in estudantes:
  inscricao_id = getPre(estudante_id,ano_lectivo_id,semestre)
  if inscricao_id:
    valida_pagamento_inscr(ano_lectivo_id, semestre, estudante_id)
    confirma_inscr(inscricao_id)
  else:
    print('log Error: Estudante nao tem pre ou ja esta Inscrito')

cmd_sql("ALTER TABLE recibo ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE inscricao_valor ENABLE TRIGGER key_check_on")

#    '07.0730.2018',
#    '07.0307.2018',
#    '07.0918.2022',
#    '07.0536.2020',
#    '07.0630.2021',
#    '07.0555.2021',

(Sucesso) SQL Update
(Sucesso) SQL Update
Inserindo dados de recibo:  06.2288.2024
(Sucesso) SQL Update
Inserindo dados na tabela recebimento (comando unico):  06.2288.2024
(Sucesso) SQL Update
Inserindo dados na tabela inscricao_valor:  06.2288.2024
(Sucesso) SQL Update
Host: https://sigeur.up.ac.mz/sigeup/public/index.php
Pedido executado com sucesso Inscrição validada com Sucesso!
Inserindo dados de recibo:  06.2289.2024
(Sucesso) SQL Update
Inserindo dados na tabela recebimento (comando unico):  06.2289.2024
(Sucesso) SQL Update
Inserindo dados na tabela inscricao_valor:  06.2289.2024
(Sucesso) SQL Update
Host: https://sigeur.up.ac.mz/sigeup/public/index.php
Pedido executado com sucesso Inscrição validada com Sucesso!
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
cmd_sql("ALTER TABLE inscricao_valor DISABLE TRIGGER key_check_on")
for i in ['06.2203.2022', '06.0485.2023']:
  #--Insere dados na tabela inscricao_valor
  sqlCmd = f"""
  WITH ReferenciaCapturada AS (
      SELECT referencia, id, ano_lectivo_id, semestre
      FROM temp_inscricao
      WHERE ano_lectivo_id ='2024' AND semestre = '1' AND confirmada = '0' AND estudante_id = '{i}'
  )
  INSERT INTO inscricao_valor (estudante_id, ano_lectivo_id, semestre, valor, trace_id, referencia, key)
                              (SELECT ti.estudante_id, ti.ano_lectivo_id, ti.semestre, r.valor, r.trace_id, r.id, (select key from recibo limit 1) FROM temp_inscricao ti INNER JOIN recibo r ON r.num_comprovativo = ti.referencia where ti.id = (select id from ReferenciaCapturada))"""
  cmd_sql(sqlCmd)

cmd_sql("ALTER TABLE inscricao_valor ENABLE TRIGGER key_check_on")

(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
def confirma_inscr2(inscricao_id):
  sqlCmd = f"""
  insert into inscricao(id, ano_lectivo_id, semestre, valor, estudante_id, usuario, data_mov, anulada, nivel_estudante, entidade, referencia, online, key)
                                                       (select id, ano_lectivo_id, semestre, valor, estudante_id, 'CRON', data_mov, anulada, nivel_estudante, entidade, referencia, '1', (select key from recibo limit 1)
                                                          from temp_inscricao where id = '{inscricao_id}')"""
  cmd_sql(sqlCmd)
  sqlCmd = f"""
  insert into pauta (id, inscricao_id, ano_lectivo_id, semestre, curso_id, disciplina_id, nome_disciplina, nivel, estudante_id, turma_id, anulada, usuario, regime_id, delegacao_id, curso_habilita, plano_id, key)
                                                      (select id, inscricao_id, ano_lectivo_id, semestre, curso_id, disciplina_id, nome_disciplina, nivel, estudante_id, turma_id, anulada, 'CRON', regime_id, delegacao_id, curso_habilita, plano_id, (select key from recibo limit 1)
                                          from temp_pauta where inscricao_id = '{inscricao_id}')"""
  cmd_sql(sqlCmd)
  sqlCmd = f"""
  insert into conta_corrente (id, estudante_id, historico, debito, trace_id, entidade, referencia_bancaria, inscricao_id, data_debito, key)
                                                              (select id, estudante_id, historico, debito, 'CRON', entidade, referencia_bancaria, inscricao_id, data_debito, (select key from recibo limit 1)
                                            from temp_conta_corrente where inscricao_id = '{inscricao_id}')"""
  cmd_sql(sqlCmd)
  cmd_sql(f"UPDATE temp_inscricao set confirmada = 1 WHERE id = '{inscricao_id}'")

##confirma_inscr2

In [ ]:
# prompt: execute a funcao valida pagamento
ano_lectivo_id = '2024'
semestre= '1'
estudantes = ['07.0819.2022']
cmd_sql("ALTER TABLE recibo DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE inscricao DISABLE TRIGGER audit_actions, DISABLE TRIGGER key_check_on")
cmd_sql("ALTER TABLE pauta DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE inscricao_valor DISABLE TRIGGER key_check_on")
cmd_sql("ALTER TABLE conta_corrente DISABLE TRIGGER key_check_on")
for estudante_id in estudantes:
  inscricao_id = getPre(estudante_id,ano_lectivo_id,semestre)
  if inscricao_id:
    valida_pagamento_inscr(ano_lectivo_id, semestre, estudante_id)
    confirma_inscr2(inscricao_id)
  else:
    print('log Error: Estudante nao tem pre ou ja esta Inscrito')

cmd_sql("ALTER TABLE recibo ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE inscricao_valor ENABLE TRIGGER key_check_on")
cmd_sql("ALTER TABLE inscricao ENABLE TRIGGER audit_actions, ENABLE TRIGGER key_check_on")
cmd_sql("ALTER TABLE pauta ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE conta_corrente ENABLE TRIGGER key_check_on")

(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
cmd_sql("ALTER TABLE recibo DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE inscricao DISABLE TRIGGER audit_actions, DISABLE TRIGGER key_check_on")
cmd_sql("ALTER TABLE pauta DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE inscricao_valor DISABLE TRIGGER key_check_on")
cmd_sql("ALTER TABLE conta_corrente DISABLE TRIGGER key_check_on")
valida_pagamento_inscr(ano_lectivo_id, semestre, estudante_id)
cmd_sql("ALTER TABLE recibo ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE inscricao_valor ENABLE TRIGGER key_check_on")
cmd_sql("ALTER TABLE inscricao ENABLE TRIGGER audit_actions, ENABLE TRIGGER key_check_on")
cmd_sql("ALTER TABLE pauta ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE conta_corrente ENABLE TRIGGER key_check_on")

(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
Inserindo dados de recibo:  04.1241.2022
(Sucesso) SQL Update
Inserindo dados na tabela recebimento (comando unico):  04.1241.2022
(Sucesso) SQL Update
Inserindo dados na tabela inscricao_valor:  04.1241.2022
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update


#Inscricao Formulario

In [ ]:
sql = f"""select e.*, r.nome regime, c.nome curso, d.nome delegacao,
            ca.nome campus, d.id delegacao_id
        from estudante e
        inner join estudante_regime r on r.id = e.regime_id
        inner join curso c on c.id = e.curso_id
        inner join campus ca on ca.id = e.campus_id
        inner join delegacao d on d.id = ca.delegacao_id
        where trim(e.id) = '{id}'"""

#Insert estudent

In [ ]:


def texto_para_dicionario(texto):
    # Dividir o texto em pares chave-valor
    pares = texto.split('&')

    # Inicializar um dicionário para armazenar os resultados
    resultado = {}

    # Iterar sobre os pares e adicioná-los ao dicionário
    for par in pares:
        chave, valor = par.split('=')
        resultado[chave] = valor

    return resultado

def date_formated(data_original):
    # Data no formato original
    #data_original = "23-06-2003"

    # Converter para o formato desejado
    data_formatada = datetime.strptime(data_original, "%d-%m-%Y").strftime("%Y-%m-%d")

    # Imprimir a data formatada
    return data_formatada


In [ ]:
# Exemplo de uso:
texto_ = "criterio=save&id=06.2287.2024&update=true&ano_ingresso=2024&apelido=PEREIRA&nomes=ISSUFO+SALIMO&pai=salimo+pereira+salimo&mae=muaziza+artur&sexo=Masculino&data_nasc=11-08-2000&cell=844490202++++++&tel_fixo=+++++++++++++++&nivel=1&email=momadezecamomade%40gmail.com&delegacao=06&campus=014&regime=01&plano=008&curso=810++&curso_habilita=&habilitacao_ingresso=medio&habilitacao_escola=escola+secundaria+handan+bin+rashid+&area_formacao=12b&nivel_ano_conclusao=&precedencias=1&certificado=1&activo=1&pais=001&provincia=08++&distrito=0822&localidade=&bairro=muahivire&quarteirao=&casa=&avenida=&nacionalidade=001&doc_ident_tipo=01&doc_ident_num=030206483605a&doc_ident_emissao=08++&doc_ident_validade=19-01-2022&nuit=&obs=ADMITIDO&declaracao=0&militar=0".replace('+', ' ')
resultado_dict = texto_para_dicionario(texto_)

# Imprimir o dicionário resultante
#print(resultado_dict)
resultado_dict['nome_completo'] = f"{resultado_dict['nomes']} {resultado_dict['apelido']}"
estudante_value = f"""('{resultado_dict['id']}', '{resultado_dict['ano_ingresso']}', '{resultado_dict['curso']}', '{resultado_dict['apelido']}',
                        '{resultado_dict['nomes']}', '{resultado_dict['nome_completo']}', '{resultado_dict['campus']}', '{resultado_dict['pai']}', '{resultado_dict['mae']}', '{resultado_dict['sexo']}', '{date_formated(resultado_dict['data_nasc'])}',
                        '{resultado_dict['regime']}', '{resultado_dict['localidade']}', '{resultado_dict['distrito']}', '{resultado_dict['provincia']}', '{resultado_dict['pais']}', '{resultado_dict['bairro']}',
                        '{resultado_dict['avenida']}', '{resultado_dict['quarteirao']}', '{resultado_dict['casa']}', '{resultado_dict['doc_ident_tipo']}', '{resultado_dict['doc_ident_num']}',
                        '{resultado_dict['doc_ident_emissao']}', '{date_formated(resultado_dict['doc_ident_validade'])}', '{resultado_dict['cell']}', '{resultado_dict['tel_fixo']}', '{resultado_dict['email']}',
                        '{resultado_dict['habilitacao_ingresso']}', '{resultado_dict['habilitacao_escola']}', '{resultado_dict['area_formacao']}', (select data_matricula from estudante limit 1), '{resultado_dict['obs']}',
                        '{resultado_dict['activo']}', '{resultado_dict['plano']}', 'import', '{resultado_dict['nivel']}', '{resultado_dict['curso_habilita']}', '{resultado_dict['nacionalidade']}')"""
sqlformat = f"""INSERT INTO estudante(
                        id, ano_ingresso, curso_id, apelido,
                        nomes, nome_completo, campus_id, pai, mae, sexo, data_nasc,
                        regime_id, localidade, distrito, provincia, pais,
                        bairro, avenida, quarteirao, casa, doc_ident_tipo, doc_ident_num,
                        doc_ident_emissao, doc_ident_validade, cell, tel_fixo, email,
                        habilitacao_ingresso, habilitacao_escola, area_formacao,
                        data_matricula, obs, activo, plano_id, trace_id, nivel, curso_habilita, nacionalidade)
                    VALUES {estudante_value}"""
cmd_sql("ALTER TABLE estudante DISABLE TRIGGER key_check_on")
cmd_sql(sqlformat)
#print(sqlformat)

data_nasc = resultado_dict['data_nasc']
sqlUser = f"""insert into usuario (id, nome, senha, activo, delegacao_id, perfil_id, tipo, trace_id, cell, key)
                                          values('{resultado_dict['id']}', '{resultado_dict['nome_completo']}', '{md5Hash(data_nasc.replace('-',''))}', '1', '{resultado_dict['delegacao']}', '015', '1', 'jadriano', '{resultado_dict['cell']}', (select key from usuario limit 1))"""
cmd_sql("ALTER TABLE usuario DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
cmd_sql(sqlUser)
cmd_sql("ALTER TABLE estudante ENABLE TRIGGER key_check_on")
cmd_sql("ALTER TABLE usuario ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")
#print(sqlUser)

(Sucesso) SQL Update
(Sucesso) SQL Update
11082000
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
sqlUser = f"""insert into usuario (id, nome, senha, activo, delegacao_id, perfil_id, tipo, trace_id, cell, key)
                                          values('{resultado_dict['id']}', '{resultado_dict['nome_completo']}', '{md5Hash(data_nasc.replace('-',''))}', '1', '{resultado_dict['delegacao']}', '015', '1', 'jadriano', '{resultado_dict['cell']}', (select key from usuario limit 1))"""
cmd_sql("ALTER TABLE usuario DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
cmd_sql(sqlUser)

01102005
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
cmd_sql("ALTER TABLE estudante ENABLE TRIGGER key_check_on")
cmd_sql("ALTER TABLE usuario ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")

(Sucesso) SQL Update
(Sucesso) SQL Update


##Update estudent

In [ ]:
user = 'fbcataua'
cmd_sql("ALTER TABLE usuario DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions, DISABLE TRIGGER set_password")
cmd_sql(f"UPDATE usuario set senha = '8b6bc13c4e85d0f549a65764fb8451c5' where id='{user}'")
#cmd_sql(f"UPDATE usuario set senha = '8b6bc13c4e85d0f549a65764fb8451c5' where id='{user}'")
#cmd_sql(f"UPDATE usuario set activo = '1' where id='{user}'")
cmd_sql("ALTER TABLE usuario ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions, ENABLE TRIGGER set_password")

(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
#(id, nome, senha, activo, delegacao_id, perfil_id, tipo, trace_id, cell, key)
texto_ = "criterio=save&id=06.2014.2024&update=true&ano_ingresso=2024&apelido=SALEHE&nomes=TANIA+FERNANDO&pai=FERNANDO+SALEHE&mae=ELISA+DANIEL&sexo=Feminino&data_nasc=16-08-2002&cell=85+++++++++++++&tel_fixo=+++++++++++++++&nivel=1&email=&delegacao=06&campus=014&regime=01&plano=008&curso=632++&curso_habilita=&habilitacao_ingresso=MEDIO&habilitacao_escola=ESCOLA+SECUNDARIA+DE+NAMPULA&area_formacao=12B&nivel_ano_conclusao=&precedencias=1&certificado=1&activo=1&pais=001&provincia=08++&distrito=0819&localidade=081904&bairro=MICOLONE+49&quarteirao=11&casa=&avenida=&nacionalidade=001&doc_ident_tipo=01&doc_ident_num=030104716143J&doc_ident_emissao=08++&doc_ident_validade=02-09-2024&nuit=&obs=SUPLENTE&declaracao=0&militar=0".replace('+', ' ')
resultado_dict = texto_para_dicionario(texto_)
resultado_dict['nome_completo'] = f"{resultado_dict['nomes']} {resultado_dict['apelido']}"
data_nasc = resultado_dict['data_nasc']
sqlUser = f"UPDATE usuario set nome = '{resultado_dict['nome_completo']}', senha = '{md5Hash(data_nasc.replace('-',''))}' WHERE id = '{resultado_dict['id']}'"
#print(sqlUser)
cmd_sql("ALTER TABLE usuario DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
cmd_sql(sqlUser)
cmd_sql("ALTER TABLE usuario ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")

16082002
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update


#Update pauta

In [ ]:
estudante_id= input('Estudante: ')
ano = input('Ano: ')
#05.0839.2022
#SELECT docente, disciplina_id, nome_disciplina, turma_id FROM pauta WHERE estudante_id = '04.1106.2019' and ano_lectivo_id = '2020' and resultado_final != 'Aprovado' and resultado_final != 'Participou';
sql_cmd =f"'UNION+SELECT+disciplina_id,docente,null,null,turma_id,nome_disciplina,null,null,null+FROM+pauta+where+ano_lectivo_id='{ano}'+and+estudante_id='{estudante_id}'+and+resultado_final+!='Aprovado' and resultado_final+!='Participou';--"
lista = SELECT(sql_cmd)
print(lista)
sql_cmd =f"'UNION+SELECT+disciplina_id,docente,null,null,turma_id,resultado_final,null,null,null+FROM+pauta+where+ano_lectivo_id='{ano}'+and+estudante_id='{estudante_id}'+and+resultado_final+!='Aprovado' and resultado_final+!='Participou';--"
lista = SELECT(sql_cmd)
print(lista)
#content = Convert(lista)
#content

Estudante: 06.0917.2018
Ano: 2021,2022
[]
[]


In [ ]:
dd_i = 'História da educação'
sql_cmd =f"'UNION+SELECT+disciplina_id,docente,null,null,turma_id,nome_disciplina,null,null,null+FROM+pauta+where+ano_lectivo_id='{ano}'+and+estudante_id='{estudante_id}'+and+nome_disciplina='{dd_i}';--"
content = Convert(SELECT(sql_cmd))
content

[]

In [ ]:
#estudante_id= input('Estudante: ')
estudante_id= '06.0022.2020'
ano = '2023' #input('Ano: ')
#consulta as diciplinas reprovadas de acordo com o ano lectivo
# defina o parametro all=true se quiser consultar todas, ex: get_diciplinas(ano, estudante_id, True)
content = get_diciplinas(ano, estudante_id)
content

[]

In [ ]:
Content=[]
cont = 0
for c in content:
  Content.append({'disciplina_id':c['disciplina_id'], 'docente':c['docente'],'turma_id':c['turma_id']})
  print(f"{cont}.disciplina={c['disciplina_id']}&docente={c['docente']}&turma={c['turma_id']}")
  cont += 1

0.disciplina=UP_ESTEC_INF_405_A_06&docente=02971&turma=2020_1_614_4_06_02_1_614B_008


In [ ]:
#test =['teste1','teste2', 'teste3','trab1','trab2','trab3','examenormal','examerec']
#ALTER TABLE pauta DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions;
cmd_sql("ALTER TABLE pauta DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions;")

(Sucesso) SQL Update


In [ ]:
def update_pauta(testes: list, notas: list, estudante):
  for i, teste in enumerate(testes):
    cmd_sql(f"UPDATE pauta set {teste} = '{notas[i]}' where estudante_id='{estudante}' and disciplina_id = '{disciplina}' and turma_id = '{turma}'")
    print(f"UPDATE pauta set {teste} = '{notas[i]}' where estudante_id='{estudante}' and disciplina_id = '{disciplina}' and turma_id = '{turma}'")

In [ ]:
#typ =f'{test[3]}'
#nota ='14'
i = 0
disciplina = f"{Content[i]['disciplina_id']}"
turma = f"{Content[i]['turma_id']}"
estudante = estudante_id
update_pauta(['examenormal'], ['2.5'], estudante)
#cmd_sql(f"UPDATE pauta set {typ} = '{nota}' where estudante_id='{estudante}' and disciplina_id = '{disciplina}' and turma_id = '{turma}'")
#UPDATE pauta set examenormal = '9.5' where estudante_id='06.0675.2021' and disciplina_id = 'UP_XX_102_A_04' and turma_id = '2022_1_721_2_06_01_1_008'

(Sucesso) SQL Update
UPDATE pauta set examenormal = '2.5' where estudante_id='06.1376.2016' and disciplina_id = 'UP_ESTEC_INF_405_A_06' and turma_id = '2020_1_614_4_06_02_1_614B_008'


In [ ]:
cmd_sql("ALTER TABLE pauta ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions;")

(Sucesso) SQL Update


In [ ]:
#UPDATE pauta set examerec = Null where estudante_id = '04.1106.2019' AND disciplina_id= 'UP_ESCOG_M1_402_A_06' AND turma_id = '2023_2_761_4_04_02_1_741A_008' AND docente = '05956';
#UPDATE pauta set resultado_final = 'Participou' where estudante_id = '04.1106.2019' AND disciplina_id= 'UP_ESCOG_M1_402_A_06' AND turma_id = '2023_2_761_4_04_02_1_741A_008' AND docente = '05956';
i = 2
disciplina = f"{Content[i]['disciplina_id']}"
turma = f"{Content[i]['turma_id']}"
estudante = estudante_id
cmd_sql('ALTER TABLE pauta DISABLE TRIGGER trg_pauta_bu')
cmd_sql(f"UPDATE pauta set resultado_final = 'Participou' where  estudante_id='{estudante}' AND  disciplina_id = '{disciplina}' AND turma_id = '{turma}'")
cmd_sql('ALTER TABLE pauta ENABLE TRIGGER trg_pauta_bu')

(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
#UPDATE pauta set examerec = Null where estudante_id = '04.1106.2019' AND disciplina_id= 'UP_ESCOG_M1_402_A_06' AND turma_id = '2023_2_761_4_04_02_1_741A_008' AND docente = '05956';
#UPDATE pauta set resultado_final = 'Participou' where estudante_id = '04.1106.2019' AND disciplina_id= 'UP_ESCOG_M1_402_A_06' AND turma_id = '2023_2_761_4_04_02_1_741A_008' AND docente = '05956';
cmd_sql('ALTER TABLE pauta DISABLE TRIGGER trg_pauta_bu')
cmd_sql(f"UPDATE pauta set  teste2 = Null where estudante_id='{estudante}' and disciplina_id = '{disciplina}' and turma_id = '{turma}'")
cmd_sql('ALTER TABLE pauta ENABLE TRIGGER trg_pauta_bu')

(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
#Habilita as TRIGGERs key_check_on e audit_actions
cmd_sql("ALTER TABLE pauta ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")

(Sucesso) SQL Update


##try pauta

In [ ]:
#SELECT e.nomes, p.estudante_id, p.teste1, p.teste2, p.teste3, p.trab1, p.trab2, p.trab3, p.mediafreq, p.examenormal, p.examerec, p.mediafinal, p.resultado_final, p.last_update_nota
#FROM pauta p INNER JOIN estudante e ON e.id = p.estudante_id
#WHERE disciplina_id= 'UP_ESCOG_M1_402_A_06' AND turma_id = '2023_2_761_4_04_02_1_741A_008' ORDER BY estudante_id ASC;
def print_pauta(teste1, teste2, teste3):
  sql_cmd =f"'UNION+SELECT+e.nomes,p.estudante_id,{teste1},null,{teste2},{teste3},null,null,null+FROM+pauta p INNER JOIN estudante e ON e.id = p.estudante_id WHERE disciplina_id= '{disciplina}' AND turma_id = '{turma}';--"
  lista = SELECT(sql_cmd)
  return lista

i = 1
disciplina = f"{Content[i]['disciplina_id']}"
turma = f"{Content[i]['turma_id']}"
pauta_lst = print_pauta('null', 'teste2', 'teste1')
pauta_lst

In [ ]:
sql_cmd =f"'UNION+SELECT+id,null,null,null,null,null,null,null,null+FROM+estudante WHERE ano_ingresso= '2024' AND curso_id = '00220';--"
SELECT(sql_cmd)

In [ ]:
from tabulate import tabulate

# Definindo a lista
pauta_lst = print_pauta('teste1', 'trab1', 'trab2')

#Convertendo a lista em tabela
tabela = tabulate(pauta_lst, headers=["Nome", "Estudante_id", "Teste 1", "Teste 2"], tablefmt="grid")
print(tabela)

In [ ]:
import pandas as pd

# Definindo a lista
pauta_lst = print_pauta('teste1', 'trab1', 'trab2')

# Criando um DataFrame
df = pd.DataFrame(data=[pauta_lst[i:i+4] for i in range(0, len(pauta_lst), 4)],
                 columns=["Nome", "Estudante_id", "Teste 1", "Teste 2"])

# Imprimindo a tabela
print(df.to_string())

In [ ]:
def SELECT_PAUTA(consulta):

  data = f"inicio=0&id=&nome=&campus=&curso={consulta};--"

  response = requests.post(
      f'https://{host}/edondzo/view/estudante_lista.php',
      cookies=cookies,
      headers=headers,
      data=data,
      verify=False,
  )
  soup = BeautifulSoup(response.text, "html.parser")
  p_tag = soup.find_all('td')
  #print(soup)
  dados = []
  for p in p_tag:
    if p.getText() != '':
        dados.append(p.getText())

  del dados[-1]

  try:
    #print(type(dados))
    return dados
  except Exception as e:
    print('Log:', e)

In [ ]:
#sql_cmd =f"'UNION+SELECT+e.nomes,p.estudante_id,teste1,null,teste2,teste3,null,null,null+FROM+pauta p INNER JOIN estudante e ON e.id = p.estudante_id WHERE disciplina_id= '{disciplina}' AND turma_id = '{turma}';--"
sql_cmd =f"'UNION+SELECT+'nomes','estudante_id','teste1',null,'teste2','teste3',null,null,null+FROM+pauta p INNER JOIN estudante e ON e.id = p.estudante_id WHERE disciplina_id= 'UP_XX_102_A_04' AND turma_id = '2022_1_721_2_06_01_1_008';--"
lista = SELECT_PAUTA(sql_cmd)
lista

[]

##imprime pauta relatorio

In [ ]:
import requests

cookies = {
    '_ga': 'GA1.3.1878032596.1702583743',
    'PHPSESSID': '2bsbelgkt1r0nfvok2i3k2ut49',
    'twk_uuid_583f2e9e4160416f6d9678a7': '%7B%22uuid%22%3A%221.6AqqI3Y7axOheDs15X5S0uHMRLKDQGueGSfmFDOUa4HGFBSsNMXCyb75LuaZRiRhdCQPcVEDnZwza8DVCjB2YM9R8OXJxMEHrJcFFx1H72d6T9mG%22%2C%22version%22%3A3%2C%22domain%22%3A%22up.ac.mz%22%2C%22ts%22%3A1708937879481%7D',
    '_gid': 'GA1.3.1271892707.1709402741',
    'TawkConnectionTime': '0',
    'twk_uuid_587de8aa1855346ccc7c3a04': '%7B%22uuid%22%3A%221.6Aqi7EKIjNgiMmmBplN4DXLsRiAeWDybrno7DvIaZHv8r2Z1xY7Qc9a2v0NeeHpgYGKeYaK0AlpyB1a7tZbZZfguOGcu2H9lMGzi0c45hePrdoQf%22%2C%22version%22%3A3%2C%22domain%22%3A%22up.ac.mz%22%2C%22ts%22%3A1709402842532%7D',
}

headers = {
    'Host': 'sigeur.up.ac.mz',
    # 'Content-Length': '129',
    'Cache-Control': 'max-age=0',
    'Sec-Ch-Ua': '"Chromium";v="119", "Not?A_Brand";v="24"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.199 Safari/537.36',
    'Origin': 'https://sigeur.up.ac.mz',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://sigeur.up.ac.mz/sigeup/portal.php',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-PT,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Priority': 'u=0, i',
    'Connection': 'close',
    # 'Cookie': '_ga=GA1.3.1878032596.1702583743; PHPSESSID=2bsbelgkt1r0nfvok2i3k2ut49; twk_uuid_583f2e9e4160416f6d9678a7=%7B%22uuid%22%3A%221.6AqqI3Y7axOheDs15X5S0uHMRLKDQGueGSfmFDOUa4HGFBSsNMXCyb75LuaZRiRhdCQPcVEDnZwza8DVCjB2YM9R8OXJxMEHrJcFFx1H72d6T9mG%22%2C%22version%22%3A3%2C%22domain%22%3A%22up.ac.mz%22%2C%22ts%22%3A1708937879481%7D; _gid=GA1.3.1271892707.1709402741; TawkConnectionTime=0; twk_uuid_587de8aa1855346ccc7c3a04=%7B%22uuid%22%3A%221.6Aqi7EKIjNgiMmmBplN4DXLsRiAeWDybrno7DvIaZHv8r2Z1xY7Qc9a2v0NeeHpgYGKeYaK0AlpyB1a7tZbZZfguOGcu2H9lMGzi0c45hePrdoQf%22%2C%22version%22%3A3%2C%22domain%22%3A%22up.ac.mz%22%2C%22ts%22%3A1709402842532%7D',
}

data = {
    'docente': '05406',
    'disciplina': 'UP_ESCOG_ECO_201_A_04',
    'nome_disciplina': 'Contabilidade Analítica',
    'turma': '2023_2_721_2_06_01_1_008',
    'tipo': '1',
}

response = requests.post(
    'https://sigeur.up.ac.mz/sigeup/reports/pauta_oficial.php',
    cookies=cookies,
    headers=headers,
    data=data,
    verify=False,
)


In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
import PyPDF2
def read_response_pdf():

    # Extrair o conteúdo da resposta
    pdf_content = response.content

    # Salvar o conteúdo em um arquivo PDF
    with open("resposta.pdf", "wb") as f:
        f.write(pdf_content)

    # Abrir o arquivo PDF
    pdf_reader = PyPDF2.PdfReader("resposta.pdf")

    # Obter o número de páginas
    num_pages = len(pdf_reader.pages)

    # Extrair o texto da primeira página
    page_object = pdf_reader.pages[0]
    text = page_object.extract_text()

    # Imprimir o texto
    print(text)

read_response_pdf()

UNIVERSIDADE ROVUMA
Faculdade de Economia e Gestão
Departamento de Economia e Gestão
Curso de Licenciatura em Economia
Disciplina: Contabilidade Analítica
Plano: PLANO B
Ano Lectivo: 2023Semestre: 2Ano: 2Turma: 1Créditos: ___Regime: Laboral
Docente: LUCIA ROSáRIO PULIVEIRA
* M.HC: Média da avaliação das Horas de Contacto = 75%
        M.HEI: Média da avaliação das Horas de Estudo Independente = 25%
        M.F: Média de Frequência = 75%
*NOTAS DE FREQUÊNCIA EXAME Nota Resultado
N.ºNº Estudante Nome do Estudante M.HCM.HEI M.F Resultado Nor RecFinal Final
106.1402.2022 AINA JUMA 131113Admitido       11.5 13Aprovado
206.1404.2022 AMINA SALIMO GELANE 161416Dispensado     16Aprovado
306.2300.2022 ANCHA ABIBE MAHADAL 101110Admitido       11 10Aprovado
406.1405.2022 ANDRE AMARAL SANTOMEIO 141113Admitido       710.512Aprovado
506.1406.2022 BERNARDO LUIS JUNIOR 101010Admitido       6.54.5 Reprovado
606.2189.2022 CATIANA ALFREDO 111412Admitido       3.5 1012Aprovado
706.2190.2022 CHAQUILA MAHAMU

#Conta corrente

In [ ]:
estudante_id = '05.0001.2008'
#06.0439.2019
sum_debito(years=list(range(int(estudante_id[8:]),2025)), estudante_id=estudante_id)

['1240.00']


In [ ]:
print(list(range(2013, 2023)))

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


In [ ]:
#sql_cmd =f"'UNION+SELECT+CAST(SUM(debito) AS VARCHAR),null,null,null,null,null,null,null,null+FROM conta_corrente WHERE extract(year from data_debito) in {tuple(years)} AND extract(month from data_debito) <= 12 AND upper(estudante_id) = '{estudante_id}' AND CAST(credito AS varchar) != CAST(debito AS varchar);--"
#years=['2023', '2024']
consulta = f"'UNION+SELECT+column_name,null,null,null,null,null,null,null,null+FROM information_schema.columns WHERE table_name='conta_corrente';--"
SELECT(consulta)

In [ ]:
propids = getListProp(years=list(range(int(estudante_id[8:]),2025)), estudante_id=estudante_id) #set h ='historico', , value=True
propids

['00000000000006857194', '00000000000006087399', '00000000000006213724']

In [ ]:
"""propids = ['00000000000006827616',
 '00000000000006809658',
 '00000000000006852108',
 '00000000000006832718',
 '00000000000006838659',
 '00000000000006809655',
 '00000000000006820032',
 '00000000000006855564',
 '00000000000006809656',
 '00000000000006809659',
 '00000000000006817706',
 '00000000000006824927',
 '00000000000006830002',
 '00000000000006841969',
 '00000000000006809657',
 '00000000000006835595',
 '00000000000006845279',
 '00000000000006822221',
 '00000000000006848652']

 estudante_id = '05.0509.2020'
  ['00000000000006201700',
 '00000000000006201697',
 '00000000000006201699',
 '00000000000006201698',
 '00000000000006201701']

 estudante_id = '04.0073.2022'
 ['00000000000006129998',
 '00000000000006129999',
 '00000000000006130000',
 '00000000000006129996',
 '00000000000006129997']

 estudante_id = '07.0057.2013'
 ['00000000000005906614',
 '00000000000005906618',
 '00000000000005906616',
 '00000000000005906615',
 '00000000000005906617']

 estudante_id = '07.0505.2021'
 ['00000000000006798263',
 '00000000000006798260',
 '00000000000006798261',
 '00000000000006798259',
 '00000000000006798262']


 ['00000000000006883080',
 'Propina Julho 2024',
 '00000000000006903629',
 '3ª Multa da Propina Abril 2024',
 '00000000000006891298',
 '1ª Multa da Propina Março 2024',
 '00000000000006893962',
 '2ª Multa da Propina Março 2024',
 '00000000000006883078',
 'Propina Abril 2024',
 '00000000000006934262',
 '2ª Multa da Propina Maio de 2024',
 '00000000000006926824',
 '3ª Multa da Propina Junho 2024',
 '00000000000006883077',
 'Propina Junho 2024',
 '00000000000006901134',
 '2ª Multa da Propina Abril 2024',
 '00000000000006907353',
 '1ª Multa da Propina Julho 2024',
 '00000000000006089807',
 '1ª Multa da Propina Julho 2021',
 '00000000000006911260',
 '2ª Multa da Propina Julho 2024',
 '00000000000006922940',
 '2ª Multa da Propina Junho 2024',
 '00000000000006896174',
 '3ª Multa da Propina Março 2024',
 '00000000000006930561',
 '1ª Multa da Propina Maio de 2024',
 '00000000000006883079',
 'Propina Maio de 2024',
 '00000000000006075202',
 '1ª Multa da Propina Junho 2021',
 '00000000000006937963',
 '3ª Multa da Propina Maio de 2024',
 '00000000000006898639',
 '1ª Multa da Propina Abril 2024',
 '00000000000006883081',
 'Propina Março 2024',
 '00000000000006915167',
 '3ª Multa da Propina Julho 2024',
 '00000000000006919056',
 '1ª Multa da Propina Junho 2024']"""



#['19065.00'], estudante_id = '07.0363.2021'
propids = ['00000000000006930561']

In [ ]:
print(5580*3)

16740


In [ ]:
year = '2024'
#estudante_id = '08.0189.2015'
cmd_sql("ALTER TABLE recibo DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
for pid in propids:
  update_conta_corrente(pid, estudante_id, year, '1')
cmd_sql("ALTER TABLE recibo ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")

(Sucesso) SQL Update
Inserindo dados de recibo:  00000000000006857194
(Sucesso) SQL Update
Inserindo dados na tabela recebimento (comando unico):  00000000000006857194
(Sucesso) SQL Update
Inserindo dados de recibo:  00000000000006087399
(Sucesso) SQL Update
Inserindo dados na tabela recebimento (comando unico):  00000000000006087399
(Erro ao gravar) SQL Update: ['Imposs\\u00edvel eliminar INSTITUI\\u00c7\\u00c3O:<\\/h4>SQLSTATE[23505]: Unique violation: 7 ERROR:  duplicate key value violates unique constraint \\"recebimento_pk\\"\\nDETAIL:  Key (id)=(20230000000007389240 ) already exists. \\r\\n                      <\\/div>"']
Inserindo dados de recibo:  00000000000006213724
(Sucesso) SQL Update
Inserindo dados na tabela recebimento (comando unico):  00000000000006213724
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
valida_props(year, estudante_id, propids)

In [ ]:
year = '2024'
cmd_sql("ALTER TABLE conta_corrente DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions;")
valida_props(year, estudante_id, propids)
cmd_sql("ALTER TABLE conta_corrente ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")

(Sucesso) SQL Update
(Sucesso) SQL Update
Pedido executado com sucesso, Mensalidades validadas com Sucesso!
(Sucesso) SQL Update


#Pagamento de Emolumentos

In [ ]:
sql_cmd =f"'UNION+SELECT+null,null,null,null,null,max(recibo_id),null,null,null+FROM+pagamento_online"
seq_recibo_id = SELECT(sql_cmd)[0]
seq_recibo_id

'20201100000004660645'

In [ ]:
int(seq_recibo_id)

20201100000004660645

In [ ]:
def pagamento_emolumentos(estudante_id,seq_recibo_id,ent,ref):
  cmd_sql(f"UPDATE pagamento_online set pago='1' where estudante_id='{estudante_id}' and entidade='{ent}' and referencia='{ref}' and pago='0'")
  cmd_sql(f"UPDATE pagamento_online set recibo_id='{int(seq_recibo_id)+1}' where estudante_id='{estudante_id}' and entidade='{ent}' and referencia='{ref}' and pago='0'")

In [ ]:
estudante_id = '06.0992.2020' #input('Estudante Id: ')
ent = '88041' #input('Entidade: ')
ref = '03329183013' #input('Referencia: ')

In [ ]:
pagamento_emolumentos(estudante_id,seq_recibo_id,ent,ref)

(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
20201100000004660646

# pesquisa permissao

1. Pesquisar pelo id do menu
2. pesquisar pelo id de permissao de acordo com o id do menu
3. pesquisar pelo id perfil correspondente
4. pesqusar pelos usuarios com o id do perfil

In [ ]:
#Pesquisar pelo id do menu
sql_cmd ="'UNION+SELECT+id,nome,null,null,null,null,null,null,null+FROM+menu"
result = SELECT(sql_cmd)
print(result)

In [ ]:
#pesquisar pelo id de permissao de acordo com o id do menu
sql_cmd ="'UNION+SELECT+id,nome,null,null,null,null,null,null,null+FROM+permissao where menu_id = '04'"
result = SELECT(sql_cmd)
print(result)

['407', 'Mudança de Regime', '405', 'Pesquisar Inscrição', '410', 'Pedido de Re-Ingresso', '404', 'Inscrição Exames Especiais', '408', 'Mudança de Turma', '419', 'Emissão de Declarações', '416', 'Cartão de Estudante', '409', 'Anulação Oficial da Inscrição', '421', 'Matrículas', '415', 'Gestão de Turmas', '420', 'Pesquisar Declarações', '403', 'Inscrição de Estudante', '401', 'Pesquisar Estudante', '411', 'Gestão das Notas Finais', '400', 'Registar Estudante', '418', 'Pesquisar Acta de Conclusão', '406', 'Mudança de Curso', '402', 'Gerir Estudante', '417', 'Emissão de Acta de Conclusão', '422', 'Validação de Pré-Inscrições']


In [ ]:
#pesquisar pelo id perfil correspondente
sql_cmd ="'UNION+SELECT+null,null,null,null,perfil_id,null,null,null,null+FROM+perfil_permissao+where+permissao_id = '421'"
result = SELECT(sql_cmd)
print(result)

['035', '026', '011', '027', '009', '003', '007', '001', '037']


In [ ]:
#pesqusar pelos usuarios com o id do perfil
sql_cmd ="'UNION+SELECT+id,senha,null,null,null,null,null,null,null+FROM+usuario+where+perfil_id in ('035', '026', '011', '027', '009', '003', '007', '001', '037') and activo = '1'"
users = list_to_dic(SELECT(sql_cmd))
users

{'csengo': '183269001cc842dc5fd3c213c456d941',
 'zjunior': '7002183387afa7c71d28138b95b146b8',
 'rhramos': '9043cc3075dcb6dc22f36bdefd491b2f',
 'fbcataua': '8b6bc13c4e85d0f549a65764fb8451c5',
 'ssalvado': '7002183387afa7c71d28138b95b146b8',
 'MaizerG': '518bdd128d467ae62fa157fbd73d39f8',
 'emaria': '393a13187b2e56c88e161a5008294123'}

#User

In [ ]:
#06.1779.2020
#ef63cfaa21e84650368e9cd68c08ef2c
#user = '06.2239.2024'
def reset_senha(user):
  cmd_sql("ALTER TABLE usuario DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions, DISABLE TRIGGER set_password")
  cmd_sql(f"UPDATE usuario set senha = '{md5Hash(user)}' where id='{user}'")
  #cmd_sql(f"UPDATE usuario set senha = 'ef63cfaa21e84650368e9cd68c08ef2c' where id='{user}'")
  cmd_sql(f"UPDATE usuario set activo = '1' where id='{user}'")
  cmd_sql("ALTER TABLE usuario ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions, ENABLE TRIGGER set_password")

In [ ]:
users = input("Insira os usuario, separe com virgula: ").split(",")
for u in users:
  reset_senha(u)

Insira os usuario, separe com virgula: 06.0056.2022
(Sucesso) SQL Update
06.0056.2022
(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
#SELECT e.activo estudante_a, u.activo user_a, u.senha FROM estudante e INNER JOIN usuario u ON u.id = e.id WHERE e.id = '06.1779.2020';
sql_cmd ="'UNION+SELECT+u.senha,null,null,null,null,null,null,null,null+FROM+usuario u WHERE id = 'tferreira'"
result = SELECT(sql_cmd)
print(result)

['63d02c1a38c64d450fbe2f40e8352fdb']


In [ ]:
a = 2
b = 5
for i in [a, b]:
  print(i)

2
5


#Divida

In [ ]:
#consulta dividas de inscricao
estudante_id = '06.0910.2020'
sql_cmd =f"'UNION+SELECT+i.id,CAST(i.ano_lectivo_id AS VARCHAR),null,null,CAST(i.semestre AS VARCHAR),null,i.referencia,CAST(i.valor as varchar),CAST(i.entidade AS VARCHAR)+FROM+inscricao i LEFT JOIN recibo r ON r.num_comprovativo = i.referencia WHERE i.estudante_id = '{estudante_id}' AND r.num_comprovativo IS NULL;"
result1 = SELECT(sql_cmd)
print(result1)

#consulta dividas de conta_corrente
estudante_id = '06.2285.2017'
sql_cmd =f"'UNION+SELECT+c.id,CAST(c.referencia_bancaria AS VARCHAR),NULL,null,null,null,CAST(c.debito AS VARCHAR),null,null+FROM+conta_corrente c LEFT JOIN recibo r ON r.num_comprovativo = c.referencia_bancaria WHERE c.estudante_id = '{estudante_id}' AND r.num_comprovativo IS NULL;"
#result = SELECT(sql_cmd)
#print(result)

['00000000000001623925', '2023', '1', '04373520171', '4550.00', '00000000000001612624', '2022', '2', '04270821216', '4550.00', '00000000000001556795', '2022', '1', '04011310157', '4550.00', '00000000000001422811', '2021', '1', '03690460179', '6350.00', '00000000000001494566', '2021', '2', '03690461262', '4550.00']


In [ ]:
def getRef(ano, semestre):
  sql_cmd =f"'UNION+SELECT+i.referencia,null,null,null,null,null,null,null,null+FROM+inscricao i LEFT JOIN recibo r ON r.num_comprovativo = i.referencia WHERE i.ano_lectivo_id = '{ano}' and i.semestre = '{semestre}' and r.num_comprovativo IS NULL;"
  result1 = SELECT(sql_cmd)
  if result1:
    return result1[0]
getRef('2022', '1')

'04011310157'

In [ ]:
def update_value(id, value = 4550):
  cmd_sql("ALTER TABLE inscricao DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
  cmd_sql(f"UPDATE inscricao set valor = {value} where id = '{id}'")
  cmd_sql("ALTER TABLE inscricao ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")

def update_ref(id, ref):
  cmd_sql("ALTER TABLE inscricao DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
  cmd_sql(f"UPDATE inscricao set referencia = '{ref}' where id = '{id}'")
  cmd_sql("ALTER TABLE inscricao ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")

In [ ]:
update_ref('00000000000001612624', getRef('2022', '2'))

(Sucesso) SQL Update
(Sucesso) SQL Update
(Sucesso) SQL Update


In [ ]:
def getPreviwID(ano, semestre):
  sql_cmd =f"'UNION+SELECT+r.id,null,null,null,null,null,null,null,null+FROM+recibo r WHERE num_comprovativo = (SELECT referencia FROM inscricao i INNER JOIN recibo r ON i.referencia = r.num_comprovativo WHERE ano_lectivo_id = '{ano}' AND semestre = '{semestre}' LIMIT 1);"
  return SELECT(sql_cmd)[0]

In [ ]:
total = []
for i in range(0, len(result1), 5):
  divida_total= {}
  divida_total['id'] = result1[i]
  divida_total['ano'] = result1[i+1]
  divida_total['sem'] = result1[i+2]
  divida_total['ref'] = result1[i+3]
  divida_total['valor'] = result1[i+4]
  total.append(divida_total)
  #print(total)
total

[{'id': '00000000000001623925',
  'ano': '2023',
  'sem': '1',
  'ref': '04373520171',
  'valor': '4550.00'},
 {'id': '00000000000001612624',
  'ano': '2022',
  'sem': '2',
  'ref': '04270821216',
  'valor': '4550.00'},
 {'id': '00000000000001556795',
  'ano': '2022',
  'sem': '1',
  'ref': '04011310157',
  'valor': '4550.00'},
 {'id': '00000000000001422811',
  'ano': '2021',
  'sem': '1',
  'ref': '03690460179',
  'valor': '6350.00'},
 {'id': '00000000000001494566',
  'ano': '2021',
  'sem': '2',
  'ref': '03690461262',
  'valor': '4550.00'}]

In [ ]:
for d in total:
  print(d['valor'] == '0.00')
  if d['valor'] == '0.00':
    update_value(id=d['id'])

20200600000004157536
DO $$
DECLARE
    v_sequencia bigint;
    v_id text;
    v_recibo record;
BEGIN
    -- Inicialize o valor da sequência
    v_sequencia := 4157536;
    
    LOOP
        -- Construa o ID com o prefixo e a sequência
        v_id := '2020060000000' || v_sequencia;
        
        -- Realize a consulta com o ID atual
        SELECT * INTO v_recibo FROM recibo WHERE id = v_id;
        
        -- Verifique se o resultado é nulo
        IF v_recibo IS NULL THEN
            RAISE NOTICE 'ID NULL: %', v_id;
            EXIT;
        END IF;
            
        -- Incremente a sequência para a próxima consulta
        v_sequencia := v_sequencia + 1;
    END LOOP;
END $$;

In [ ]:
#v_sequencia = 4421270
def next_recibo_id(v_sequencia: int) -> str | None:
  while True:
    query = f"'UNION+SELECT+id,null,null,null,null,null,null,null,null+FROM+recibo WHERE id = '{v_sequencia}'"
    result = SELECT(query)
    if len(result) == 0:
      break
    else:
      v_sequencia += 1
  print('New recibo ID:', v_sequencia)
  return v_sequencia

In [ ]:
def pagamento_divida(recibo_id, dateDivida, estudante_id):
  #INSERT INTO recibo
  INSERT_INTO_RECIBO = f"""
  WITH DatasCapturada AS (
      SELECT data_comprovativo, data_emissao, trace_id
      FROM recibo
      WHERE num_comprovativo = (SELECT referencia FROM inscricao i
          INNER JOIN recibo r ON i.referencia = r.num_comprovativo
          WHERE ano_lectivo_id ='{dateDivida['ano']}' AND semestre = '{dateDivida['sem']}'
          LIMIT 1)
  )
  INSERT INTO recibo (id, estudante_id, data_emissao, banco, forma_pagto, num_comprovativo, data_comprovativo, valor, extenso, trace_id)
  SELECT '{recibo_id}', i.estudante_id, (SELECT data_emissao FROM DatasCapturada), (SELECT MAX(banco) FROM recibo WHERE estudante_id = '{estudante_id}'), 'Servicos', i.referencia, (SELECT data_comprovativo FROM DatasCapturada), i.valor, (select extenso from recibo where valor = i.valor limit 1), (select trace_id from DatasCapturada)
  FROM inscricao i
  WHERE i.referencia = '{dateDivida['ref']}'"""
  print('Inserido dados de recibo')
  cmd_sql(INSERT_INTO_RECIBO)

  #insere dados na tabela recebimento (comando unico)
  INSERT_INTO_RECEBIMENTO = f"""WITH ReciboCapturada as (
        SELECT r.id, r.trace_id
          FROM recibo r
          INNER JOIN inscricao i ON i.referencia = r.num_comprovativo
          WHERE i.ano_lectivo_id ='{dateDivida['ano']}' AND i.semestre = '{dateDivida['sem']}' AND i.estudante_id = '{estudante_id}'
      ),
      DataCapturada AS (
          SELECT data_movimento
          FROM recebimento
          WHERE recibo_id = (SELECT id FROM recibo where num_comprovativo = (SELECT referencia FROM inscricao i
              INNER JOIN recibo r ON i.referencia = r.num_comprovativo
              WHERE ano_lectivo_id = '{dateDivida['ano']}' AND semestre = '{dateDivida['sem']}' LIMIT 1)) LIMIT 1
      )
      INSERT INTO recebimento (id, tipo, estudante_id, historico, valor, recibo_id, trace_id, data_movimento)
      SELECT '2022000000000' || (SELECT tp.id::text), 'Emolumento', i.estudante_id, tp.nome, tp.valor, (select id from ReciboCapturada), (select trace_id from ReciboCapturada), (select data_movimento from DataCapturada) FROM inscricao i, temp_pagamentos tp where i.referencia = '{dateDivida['ref']}' AND tp.temp_inscricao_id = '{dateDivida['id']}'"""
  print('Inserindo dados de recebimento')
  cmd_sql(INSERT_INTO_RECEBIMENTO)
  #INSERT INTO inscricao_valor
  inscricao_valor = f"""INSERT INTO inscricao_valor (estudante_id, ano_lectivo_id, semestre, valor, trace_id, referencia, key)
                            (SELECT ti.estudante_id, ti.ano_lectivo_id, ti.semestre, r.valor, r.trace_id, r.id, (select key from recibo limit 1) FROM temp_inscricao ti INNER JOIN recibo r ON r.num_comprovativo = ti.referencia where ti.id = '{dateDivida['id']}')"""
  print('Inserindo dados de valor')
  cmd_sql(inscricao_valor)

In [ ]:
cmd_sql("ALTER TABLE recibo DISABLE TRIGGER key_check_on, DISABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE inscricao_valor DISABLE TRIGGER key_check_on")
for dd in total:
  seq_recibo_id = getPreviwID(dd['ano'], dd['sem'])
  pagamento_divida(recibo_id=next_recibo_id(v_sequencia=int(seq_recibo_id)), dateDivida=dd, estudante_id=estudante_id)
cmd_sql("ALTER TABLE recibo ENABLE TRIGGER key_check_on, ENABLE TRIGGER audit_actions")
cmd_sql("ALTER TABLE inscricao_valor ENABLE TRIGGER key_check_on")

(Sucesso) SQL Update
(Sucesso) SQL Update
